In [1]:
import yaml
import time
t0 = time.time()
# - - - Get DataBases - Init - - -
with open('rec\\sde\\fsd\\blueprints.yaml', "r") as stream:
    try:
        _BPData = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)
        exit()
with open('rec\\sde\\fsd\\typeIDs.yaml', "r" , encoding="utf8") as stream:
    try:
        _NameData = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)
        exit()
print("Time to execute: " + str(time.time() - t0) + " seconds")

Time to execute: 131.69347071647644 seconds


In [3]:
def getBluePrintID(ItemID):
    for x in _BPData:
        if "manufacturing" in _BPData[x]["activities"]:
            if "products" in _BPData[x]["activities"]["manufacturing"]:
                if _BPData[x]["activities"]["manufacturing"]["products"][0]["typeID"] == ItemID:
                    #print(str(ItemID) + " is an Manufactured Item and has the BP ID: " + str(x))
                    return x, 1

        if "reaction" in _BPData[x]["activities"]:
            if _BPData[x]["activities"]["reaction"]["products"][0]['typeID'] == ItemID:
                if _BPData[x]["activities"]["reaction"]["products"][0]["typeID"] == ItemID:
                    #print(str(ItemID) + " is an Reaction Item and has the BP ID: " + str(x))
                    return x, 2

    #print("Didnt find BP")
    return 0, 0 # error, didnt find anything!

In [4]:
def readBlueprint(ID, mantype):  
    print(_NameData[ID]["name"]["en"] + "\n")
    
    if(mantype == 1):
        for x in _BPData[ID]["activities"]["manufacturing"]["materials"]:
            _BPId, _mantype = getBluePrintID(x["typeID"])      
            if(_BPId == 0):
                print(str(_NameData[x["typeID"]]["name"]["en"]) + ": " + str(x["quantity"]) )#+ " -ID davon: " + str(x["typeID"]))
                continue
            print(str(_NameData[x["typeID"]]["name"]["en"]) + ": " + str(x["quantity"]) )
            readBlueprint(_BPId, _mantype)
    if(mantype == 2):
        for x in _BPData[ID]["activities"]["reaction"]["materials"]:
            _BPId, _mantype = getBluePrintID(x["typeID"])        
            if(_BPId == 0):
                print(str(_NameData[x["typeID"]]["name"]["en"]) + ": " + str(x["quantity"]) )#+ " -ID davon: " + str(x["typeID"]))
                #print(getBluePrintID(x))
                continue
            print(str(_NameData[x["typeID"]]["name"]["en"]) + ": " + str(x["quantity"]) )
            readBlueprint(_BPId, _mantype)
    if(mantype == 0): 
        print("Didnt find production type of " + str(ID) + " as the detected prod. type is: " + str(mantype))
    return

In [2]:
def main(): 
    # Raven - 688
    # Cormorant - 16239
    #readBlueprint(16239, 1)
    BPID, MANTYPE = getBluePrintID(638)
    readBlueprint(BPID, MANTYPE)
# - - -

In [5]:
if __name__ == "__main__":
    main()

Raven Blueprint

Tritanium: 8000000
Pyerite: 4000000
Mexallon: 600000
Isogen: 400000
Nocxium: 12000
Zydrine: 6000
Megacyte: 3000
Auto-Integrity Preservation Seal: 200
Auto-Integrity Preservation Seal Blueprint

Bacteria: 80
Proteins: 40
Reinforced Carbon Fiber: 10
Reinforced Carbon Fiber Reaction Formula

Carbon Fiber: 200
Carbon Fiber Reaction Formula

Hydrogen Fuel Block: 5
Hydrogen Fuel Block Blueprint

Robotics: 1
Enriched Uranium: 4
Mechanical Parts: 4
Coolant: 9
Strontium Clathrates: 20
Oxygen: 22
Heavy Water: 170
Liquid Ozone: 350
Hydrogen Isotopes: 450
Hydrocarbons: 100
Evaporite Deposits: 100
Oxy-Organic Solvents: 1
Oxy-Organic Solvents Reaction Formula

Oxygen Fuel Block: 5
Oxygen Fuel Block Blueprint

Robotics: 1
Enriched Uranium: 4
Mechanical Parts: 4
Coolant: 9
Strontium Clathrates: 20
Oxygen: 22
Heavy Water: 170
Liquid Ozone: 350
Oxygen Isotopes: 450
Hydrocarbons: 2000
Atmospheric Gases: 2000
Thermosetting Polymer: 200
Thermosetting Polymer Reaction Formula

Oxygen Fuel B

In [8]:
def getBPArr(bluePrintID):
#gets all the Blueprints needed for a Item to be constructed...
    BPList = []
    if(getBluePrintID(bluePrintID) != (0,0)):
        theBPInstead, mantype = getBluePrintID(bluePrintID)
        BPList.append(theBPInstead)
    else:        
        BPList.append(bluePrintID)
    if(len(BPList)==0):
        return None
    skip = BPList[0]
#now we have a Blueprint in the array at pos 0 (this is the to be produced bp)
#everything else is a bp that is needed for the componence or the componence of the componence...
    for key, values in _BPData[skip].items():
        if("activities" == key):    
            if("manufacturing" in values):
                for x in values["manufacturing"]["materials"]:
                    if(getBluePrintID(x["typeID"]) == (0, 0)): #checks if has a bp
                        continue    #if not, continue
                    bpID, manu = getBluePrintID(x["typeID"])
                    BPList.append(bpID) # if yes... I have to find BP!
            if("reaction" in values):
                for x in values["reaction"]["materials"]:
                    if(getBluePrintID(x["typeID"]) == (0, 0)):
                        continue
                    bpID, manu = getBluePrintID(x["typeID"])
                    BPList.append(bpID)
#now we have the blueprints the item consists of...
    for x in BPList[1:]:
        arr = getBPArr(x)
        for y in arr:
            if y not in BPList:
                BPList.append(y)            
    return BPList

Time to execute: 0.0 seconds


In [7]:
t0 = time.time()
BPID, MANTYPE = getBluePrintID(638) #57478 | 638
bla = getBPArr(BPID)
for x in bla:
    print(_NameData[x]["name"]["en"])
print("Time to execute: " + str(time.time() - t0) + " seconds")

Raven Blueprint
Auto-Integrity Preservation Seal Blueprint
Core Temperature Regulator Blueprint
Life Support Backup Unit Blueprint
Reinforced Carbon Fiber Reaction Formula
Carbon Fiber Reaction Formula
Oxy-Organic Solvents Reaction Formula
Thermosetting Polymer Reaction Formula
Hydrogen Fuel Block Blueprint
Oxygen Fuel Block Blueprint
Pressurized Oxidizers Reaction Formula
Carbon Polymers Reaction Formula
Sulfuric Acid Reaction Formula
Helium Fuel Block Blueprint
Nitrogen Fuel Block Blueprint
Time to execute: 0.934830904006958 seconds
